In [1]:
import numpy as np
from scipy import ndimage
from time import time
import pandas as pd
from skimage import measure
from skimage import io
import ImageAnalysis
import Morphology

from pathlib import Path
import pathlib
import os
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize_3d
import sknw
import networkx as nx

In [2]:
root_path = r"/Users/rensu/Downloads/Data/Metformin MEL/"

folders = [name for name in os.listdir(root_path) if os.path.isdir(os.path.join(root_path,name))]
try:
    folders.remove('.DS_Store')
except:
    pass

print(folders)


['Baf_1010.1']


In [3]:
# count = 0
for folder in folders:
    properties = {}

    p = os.listdir(root_path + folder)
    all_files = [x for x in p if x.endswith(".tif")]
    print(all_files)

    for f in all_files:
        # count += 1
        # if count != 55:
        #     continue;
        print("Processing: " + f)
        image = io.imread(root_path + folder + "/" + f)
        #f = plt.figure(figsize=(15,15))
        #plt.imshow(image[3])
        fileName = root_path + folder + "/" + f + "_output.xlsx"

        properties[f] = Morphology.calculateAllParameters_scales(image, image, 1, 0.3, dataExportFileName=fileName);
        # _, edgeProperties = Morphology.getSkeletonProperties(image)
        # for edgeKey in edgeProperties.keys():
        #     properties[f][edgeKey] = str(edgeProperties[edgeKey])

        # print(properties[f])


['01.tif_L1 copy.tif', '01.tif_L1.tif']
Processing: 01.tif_L1 copy.tif
92


/opt/homebrew/lib/python3.10/site-packages/skimage/measure/_regionprops.py:395: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 150161579  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  9  Error-roundoff 1.2e-14  _one-merge 8.7e-14
  _near-inside 4.4e-13  Visible-distance 2.5e-14  U-max-coplanar 2.5e-14
  Width-outside 5e-14  _wide-facet 1.5e-13  _maxoutside 1e-13

  return convex_hull_image(self.image)
/opt/homebrew/lib/python3.10/site-packages/skimage/measure/_regionprops.py:395: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected f

Processing: 01.tif_L1.tif
92


/opt/homebrew/lib/python3.10/site-packages/skimage/measure/_regionprops.py:395: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 150262421  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  9  Error-roundoff 1.2e-14  _one-merge 8.7e-14
  _near-inside 4.4e-13  Visible-distance 2.5e-14  U-max-coplanar 2.5e-14
  Width-outside 5e-14  _wide-facet 1.5e-13  _maxoutside 1e-13

  return convex_hull_image(self.image)
/opt/homebrew/lib/python3.10/site-packages/skimage/measure/_regionprops.py:395: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate    0

While executing:  | qhull i Qt
Options selected f

In [4]:
# count = 0
for folder in folders:
    properties = {}
    p = os.listdir(root_path + folder)
    all_files = [x for x in p if x.endswith(".tif")]
    print(all_files)

    filePath = root_path + folder

    for f in all_files:
        print("Load: ", filePath + "/" + f + "_output.xlsx")
        properties[folder + "/" + f] = pd.read_excel( filePath + "/" + f + "_output.xlsx")


#####
    columns = ['area', 'extent', 'filled_area',
        'aspect_ratio_major_minor', 'convex_area', 'solidity',
       'AspectRatio3D Main/Middle', 'AspectRatio3D Main/Minor',
       'AspectRatio3D Middle/Minor', 'surface_area_3D_relative',
       'volume_3D_relative', 'volume_3D_um', 'extents_3D',
       'convex_area_3D_relative', 'convex_volume_3D_relative',
       'convexity_3D_relative', 'solidy_3D_relative',
       'form_factor_3D_relative', 'sphericity'
        #  "num_nodes", "num_edge", 
       ]

    allAverages = {}
    for f in properties.keys():
        print("Processing: ", f)
        image = io.imread(root_path + "/" + f)
        image = np.array(image > 0).astype(np.uint8)
        graph = sknw.build_sknw(skeletonize_3d(image))
        averages = {'count': len(properties[f])}
        averages['num_connected_components'] = nx.number_connected_components(graph)
        averages['num_isolates'] = nx.number_of_isolates(graph)
        averages['num_selfloops'] = nx.number_of_selfloops(graph)
        averages["average_edge_length"] = graph.size(
            weight="weight")/graph.number_of_edges()
        averages["total_num_nodes"] = graph.number_of_nodes()
        averages["total_num_edges"] = graph.number_of_edges()
        for c in columns:
            try:
                averages[c] = np.average(np.ma.masked_invalid(list(properties[f][c])))
            except:
                print("Failed with average for " + c)
            # print(list(properties[f][c]))

        allAverages[f] = averages

    pd.DataFrame(allAverages).transpose().to_excel(root_path + "allAverages.xlsx")

['01.tif_L1 copy.tif', '01.tif_L1.tif']
Load:  /Users/rensu/Downloads/Data/Metformin MEL/Baf_1010.1/01.tif_L1 copy.tif_output.xlsx
Load:  /Users/rensu/Downloads/Data/Metformin MEL/Baf_1010.1/01.tif_L1.tif_output.xlsx
Processing:  Baf_1010.1/01.tif_L1 copy.tif
Failed with average for extents_3D
Processing:  Baf_1010.1/01.tif_L1.tif
Failed with average for extents_3D
